In [1]:
from thermo import *
from thermo.chemical import Chemical

In [2]:
import math
from fluids import *
from fluids.units import *


Given:  A globe type lift check valve with a wing-guided disc is requried in a 3" Schedule 40 horizontal pipe carrying 70 degF water at the rate of 80 gallons per minute.

Find:  The proper size check valve an dthe pressure drop.  The valve should be sied so that the disc is fully lifted at the specified flow.

In [3]:
Q = 80*u.gallon/u.minute

In [4]:
T = (70*u.degF).to(u.degK)

Assume atmospheric pressures

In [5]:
P = (1*u.atm).to(u.Pa)

Get the water properties from PubChem

In [6]:
water = Chemical('water', P=P.magnitude, T=T.magnitude)

# Solve using the fluids-built in functions exclusively

Only the following types of clean, new pipe have data available:
‘Plastic coated’
‘Carbon steel, honed bare’
‘Cr13, electropolished bare’
‘Cement lining’
‘Carbon steel, bare’
‘Fiberglass lining’
‘Cr13, bare’

In [7]:
d1 = nearest_pipe(NPS=2.5*u.inch, schedule='40')[1] #inner pipe dimension
d2 = nearest_pipe(NPS=3*u.inch, schedule='40')[1] #inner pipe dimension

spec_vol = 1/water.rho*u.meter**3/u.kg

rho = water.rho*u.kg/u.meter**3

#Re for fully turbulent
Re = 10**8

e = roughness_Farshad('Carbon steel, bare', D=d1)
f2 = friction_factor(Re=Re, eD=e/d1)  #friction factor for 2-1/2" pipe
 
f3 = friction_factor(Re=Re, eD=e/d2)  #friciton factor for 3" pipe


In [8]:
#Let's try a 3" lift check valve
beta = d2/d2
vmin = 40*math.sqrt((spec_vol.to(u.feet**3/u.lb)).magnitude)*u.feet/u.sec #Crane 410 A-28 for lift check valve

v3 = ((0.4085*Q/(d2.to(u.inch)**2)).magnitude)*u.feet/u.sec

vmin, v3
#Since vmin is greater than v3, we need a smaller size.

(<Quantity(5.066215328554337, 'foot / second')>,
 <Quantity(3.4725746724909237, 'foot / second')>)

In [9]:
#Let's try a 2-1/2" lift check valve
v2 = ((0.4085*Q/(d1.to(u.inch)**2)).magnitude)*u.feet/u.sec

#We have a new beta
beta = d1/d2

#Calculate the downstream pressure drop
K1 = 600*f3

K2 = (K1+beta*(0.5*(1-beta**2)+(1-beta**2)**2))/beta**4  
#note K1 is already incorporated into K2, thus this K2 is actually Ktot

delta_P = ((1.801e-5)*(K2*rho.to(u.lb/u.feet**3)*Q**2)/d2.to(u.inch)**4).magnitude*u.psi

print("Using a 2-1/2\" valve with a delta_P equal {}.".format(delta_P))

Using a 2-1/2" valve with a delta_P equal 1.9454971439787705 pound_force_per_square_inch.


# Solution as Provided by Crane 410

In [10]:
f2 = 0.018 #2-1/2 fully turbulent friction factor
f3 = 0.017 #3 fully turbulent friction factor

K1 = 600*f2

beta = d1/d2

K2 = (K1+beta*(0.5*(1-beta**2)+(1-beta**2)**2))/beta**4  

delta_P = ((1.801e-5)*(K2*rho.to(u.lb/u.feet**3)*Q**2)/d2.to(u.inch)**4).magnitude*u.psi

print("Using a 2-1/2\" valve with a delta_P equal {}.".format(delta_P))

Using a 2-1/2" valve with a delta_P equal 2.1395300100036607 pound_force_per_square_inch.


The calculation from the fluids module vs what Crane 410 gives is pretty close.